**[PT]** Português

---

**[EN]** English

# Ligando o FAUC com Wikidata

---

# Linking FAUC and Wikidata


## Wikidata: Estudantes da Universidade de Coimbra nascidos antes do século XX na Wikidata

---

## Wikidata: Students of the University of Coimbra born before the 20th century in Wikidata

In [343]:
# Id of Universidade de Coimbra
uc_qid = 'Q368643'

# Property "educated at"
p_educated = 'P69'

# Property birth date
p_birth_date = 'P569'
# Property birth place
p_birth_place = 'P19'

# Property death date
p_death_date = 'P570'
# Property death place
p_death_place = 'P20'

# Coordinates of place
p_coordiantes = 'P625'

# country
p_country = 'P27'

# Biblioteca Nacional de Portugal ID
p_bnp_id = 'P1005'


min_age = 14  #  minimum age at entry
max_age = 35  # max age at entry

# Install qwikidata library

Terminal:


```console

pip install qwikidata
pip install 'recordlinkage[all]'

```

### Obter todos os estudantes da UC, nascidos antes de 1900, com data e local de nascimento e morte

E também: id na biblioteca nacional, coordenadas do local de nascimento e outros nomes.

---

### Fetch all the students of UC born before 1900 with date and place of birth and deadth and Portuguese National Library ID

And also: Id of the national library, coordinates of place of birth and aliases.

In [344]:
# for the method of getting the coordinates see https://en.wikibooks.org/wiki/SPARQL/WIKIDATA_Precision,_Units_and_Coordinates#Coordinates
all_query = """

SELECT DISTINCT ?student ?bnp_idLabel ?studentLabel ?altLabel ?countryLabel ?birthDateLabel ?birthPlace ?birthPlaceLabel ?longitude ?latitude ?deathDateLabel ?deathPlace ?deathPlaceLabel 
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "pt". }
  {
    SELECT DISTINCT ?student ?altLabel ?bnp_id ?country ?birthDate ?birthPlace ?longitude ?latitude ?deathDate ?deathPlace WHERE {
      ?student p:P69 ?statement0.
      ?statement0 (ps:P69/(wdt:P279*)) wd:Q368643.
      ?student p:P569 ?statement_1.
      OPTIONAL { ?student p:P570/psv:P570/wikibase:timeValue ?deathDate.}
      OPTIONAL { ?student p:P27/ps:P27 ?country.}
      OPTIONAL { ?student p:P19/ps:P19 ?birthPlace.} 
      OPTIONAL { ?student p:P19/ps:P19/p:P625/psv:P625/wikibase:geoLongitude ?longitude.}
      OPTIONAL { ?student p:P19/ps:P19/p:P625/psv:P625/wikibase:geoLatitude ?latitude.}
      OPTIONAL { ?student p:P20/ps:P20 ?deathPlace.}
      OPTIONAL { ?student p:P1005/ps:P1005 ?bnp_id.}
      OPTIONAL { ?student skos:altLabel ?altLabel . FILTER (lang(?altLabel) = "pt") }
      ?statement_1 psv:P569 ?statementValue_1.
      ?statementValue_1 wikibase:timePrecision ?precision_1.
      FILTER(?precision_1 >= 11 )
      ?statementValue_1 wikibase:timeValue ?birthDate.
      FILTER(?birthDate< "+1900-01-01T00:00:00Z"^^xsd:dateTime)
    }
    ORDER BY ?studentLabel ?student
    LIMIT 1000
  }
}
"""

Query Wikidata

In [345]:
from qwikidata.sparql import return_sparql_query_results

results = return_sparql_query_results(all_query)
records = results['results']['bindings']


Show sample records

In [346]:
records[:1]

[{'student': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q94850304'},
  'altLabel': {'xml:lang': 'pt',
   'type': 'literal',
   'value': 'Levy Maria Jordão de Paiva Manso'},
  'birthPlace': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q597'},
  'longitude': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '-9.139016'},
  'latitude': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '38.708042'},
  'deathPlace': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q597'},
  'bnp_idLabel': {'type': 'literal', 'value': '54707'},
  'studentLabel': {'xml:lang': 'pt',
   'type': 'literal',
   'value': 'Levy Maria Jordão'},
  'countryLabel': {'xml:lang': 'pt', 'type': 'literal', 'value': 'Portugal'},
  'birthDateLabel': {'type': 'literal', 'value': '1831-01-09T00:00:00Z'},
  'birthPlaceLabel': {'xml:lang': 'pt', 'type': 'literal', 'value': 'Lisboa'},
  'deathDateLabel': {'typ

Create dataframe from results

In [239]:
import pandas as pd


tuples = [(  
            student.get('student',{'value':None})['value'],
            student.get('studentLabel',{'value':None})['value'],
            student.get('altLabel',{'value':None})['value'],
            student.get('bnp_idLabel',{'value':None})['value'],
            student.get('birthPlaceLabel',{'value':None})['value'],
            student.get('birthPlace',{'value':None})['value'],
            student.get('longitude',{'value':None})['value'],
            student.get('latitude',{'value':None})['value'],
            student.get('birthDateLabel',{'value':'0000-00-00'})['value'][0:10], # keep just the date, the time is normally missing
            )
            for student in records]

wikidata = pd.DataFrame(tuples, columns=['wikidata_id','name','alias','bnp_id','naturalidade','place_id','longitude','latitude','birth_date'])
wikidata.drop_duplicates(subset=['wikidata_id','name','alias'], inplace=True)
wikidata['latitude'] = pd.to_numeric(wikidata['latitude'])
wikidata['longitude'] = pd.to_numeric(wikidata['longitude'])
print(wikidata.info())
print("Number of unique wikidata entities:",len(wikidata['wikidata_id'].unique()))
print("Number of unique BNP id:",len(wikidata['bnp_id'].unique()))
wikidata.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 691 entries, 0 to 842
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   wikidata_id   691 non-null    object 
 1   name          691 non-null    object 
 2   alias         464 non-null    object 
 3   bnp_id        458 non-null    object 
 4   naturalidade  654 non-null    object 
 5   place_id      654 non-null    object 
 6   longitude     652 non-null    float64
 7   latitude      652 non-null    float64
 8   birth_date    691 non-null    object 
dtypes: float64(2), object(7)
memory usage: 54.0+ KB
None
Number of unique wikidata entities: 433
Number of unique BNP id: 259


,wikidata_id,name,alias,bnp_id,naturalidade,place_id,longitude,latitude,birth_date
0,http://www.wikidata.org/entity/Q94850304,Levy Maria Jordão,Levy Maria Jordão de Paiva Manso,54707,Lisboa,http://www.wikidata.org/entity/Q597,-9.139016,38.708042,1831-01-09
2,http://www.wikidata.org/entity/Q9619744,António Maria Vasco de Melo Silva César e Meneses,António de melo césar e meneses,537789,Lisboa,http://www.wikidata.org/entity/Q597,-9.139016,38.708042,1851-11-13
3,http://www.wikidata.org/entity/Q966168,Manuel Gonçalves Cerejeira,Gonçalves cerejeira,68690,Lousado,http://www.wikidata.org/entity/Q949303,-8.530280,41.351389,1888-11-29
4,http://www.wikidata.org/entity/Q9620291,António da Costa,António da Costa e Sousa de Macedo,105800,Lisboa,http://www.wikidata.org/entity/Q597,-9.139016,38.708042,1824-11-24
5,http://www.wikidata.org/entity/Q921705,Alexandre de Serpa Pinto,Alexandre Serpa Pinto,32584,Cinfães,http://www.wikidata.org/entity/Q861464,-8.083333,41.066667,1846-04-20


In [240]:
aliases = wikidata[wikidata['alias'].notnull()]
aliases.head()
cols = list(aliases)
cols[1]='alias'
cols[2]='name'
cols
aliases = aliases.reindex(columns=cols)
wikidata = pd.concat([wikidata,aliases])
wikidata = wikidata.reset_index(drop=True)
wikidata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   wikidata_id   1155 non-null   object 
 1   name          1155 non-null   object 
 2   alias         928 non-null    object 
 3   bnp_id        805 non-null    object 
 4   naturalidade  1105 non-null   object 
 5   place_id      1105 non-null   object 
 6   longitude     1101 non-null   float64
 7   latitude      1101 non-null   float64
 8   birth_date    1155 non-null   object 
dtypes: float64(2), object(7)
memory usage: 81.3+ KB


### FAUC: nome, naturalidade e data de entrada dos estudantes



---

### FAUC: name, place of birth and date of entry of students



In [241]:
from timelinknb.pandas import attribute_to_df
from timelinknb import get_db
from ucalumni.config import default_db



db_spec = default_db
db = get_db(db_spec)


fauc = attribute_to_df("uc-entrada",more_cols=['naturalidade'],person_info=True)
fauc.drop(['uc-entrada.date','uc-entrada.obs','naturalidade.date','naturalidade.obs'],axis=1,inplace=True)
fauc['fauc_id'] = fauc.index.values
fauc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105298 entries, 140351 to 310137
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   name          105298 non-null  object
 1   sex           105298 non-null  object
 2   uc-entrada    105298 non-null  object
 3   naturalidade  98902 non-null   object
 4   fauc_id       105298 non-null  object
dtypes: object(5)
memory usage: 8.9+ MB


In [242]:
fauc.head()

,name,sex,uc-entrada,naturalidade,fauc_id
id,,,,,
140351,António Pestana Guerreiro Camacho Aboim,m,0000-00-00,Casalinho,140351
140357,Gonçalo de Aboim,m,0000-00-00,Santarém,140357
140367,Manuel de Vargas de Aboim,m,0000-00-00,Coimbra,140367
140377,André de Sequeira Abranches,m,0000-00-00,Várzea de Meruge,140377
268681,José Alexandre de Abrantes,m,0000-00-00,Cabanas,268681


#### FAUC: juntar as coordenadas


---

#### FAUC: get the coordinates



In [243]:
places_file = 'inferences/places/osm-places.csv'
geocodes = pd.read_csv('../'+places_file)
geocodes['geo_id'] = geocodes['id'].apply(str)  # convert id from int to str
geocodes.drop('id',axis=1, inplace=True)  # we don't need the original
geocodes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6296 entries, 0 to 6295
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   naturalidade  6296 non-null   object 
 1   geocoder      6296 non-null   object 
 2   address       6296 non-null   object 
 3   city          6291 non-null   object 
 4   country       6296 non-null   object 
 5   importance    6296 non-null   float64
 6   class         6296 non-null   object 
 7   type          6296 non-null   object 
 8   latitude      6296 non-null   float64
 9   longitude     6296 non-null   float64
 10  distance      6296 non-null   float64
 11  geo_id        6296 non-null   object 
dtypes: float64(4), object(8)
memory usage: 590.4+ KB


In [244]:
# merge FAUC and coordinates
fauc = fauc.merge(geocodes,how='left',on='naturalidade')
fauc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105298 entries, 0 to 105297
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   name          105298 non-null  object 
 1   sex           105298 non-null  object 
 2   uc-entrada    105298 non-null  object 
 3   naturalidade  98902 non-null   object 
 4   fauc_id       105298 non-null  object 
 5   geocoder      73560 non-null   object 
 6   address       73560 non-null   object 
 7   city          73540 non-null   object 
 8   country       73560 non-null   object 
 9   importance    73560 non-null   float64
 10  class         73560 non-null   object 
 11  type          73560 non-null   object 
 12  latitude      73560 non-null   float64
 13  longitude     73560 non-null   float64
 14  distance      73560 non-null   float64
 15  geo_id        73560 non-null   object 
dtypes: float64(4), object(12)
memory usage: 13.7+ MB


In [245]:
fauc.latitude.fillna(0, inplace=True)
fauc.longitude.fillna(0,inplace=True)
fauc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105298 entries, 0 to 105297
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   name          105298 non-null  object 
 1   sex           105298 non-null  object 
 2   uc-entrada    105298 non-null  object 
 3   naturalidade  98902 non-null   object 
 4   fauc_id       105298 non-null  object 
 5   geocoder      73560 non-null   object 
 6   address       73560 non-null   object 
 7   city          73540 non-null   object 
 8   country       73560 non-null   object 
 9   importance    73560 non-null   float64
 10  class         73560 non-null   object 
 11  type          73560 non-null   object 
 12  latitude      105298 non-null  float64
 13  longitude     105298 non-null  float64
 14  distance      73560 non-null   float64
 15  geo_id        73560 non-null   object 
dtypes: float64(4), object(12)
memory usage: 13.7+ MB


#### Cruzar informação da Wikidata com o FAUC: comparar nome e local de nascimento, verificar datas

---

#### Record matching wikidata with FAUC: use name and place of birth, check dates



In [247]:
from timelinknb.pandas import remove_particles
from recordlinkage.preprocessing import clean

# remove particles from names convert to lower case
wikidata['nname'] = wikidata['name'].apply(remove_particles)
wikidata['nname'] = wikidata['nname'].str.lower()
fauc['nname'] = fauc['name'].apply(remove_particles)
fauc['nname'] = fauc['nname'].str.lower()

In [248]:
import recordlinkage

indexer = recordlinkage.index.SortedNeighbourhood('naturalidade','naturalidade',window=21)




In [249]:
candidates = indexer.index(wikidata,fauc)
print(len(candidates))

2270433


In [331]:
compare = recordlinkage.Compare()
# see https://www.elastic.co/guide/en/elasticsearch/reference/8.2/query-dsl-function-score-query.html#function-decay
compare.geo('latitude','longitude','latitude','longitude',
    method='linear',
    offset=10,
    label='scoord')
compare.string('naturalidade','naturalidade',
    method='levenshtein',
    threshold=0.85,
    label='snat')
compare.string('nname','nname',
    method='levenshtein',
    threshold=0.85,
    label='sname')
features = compare.compute(candidates,wikidata,fauc)
features.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2270433 entries, (0, 84314) to (685, 65179)
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   scoord  float64
 1   snat    float64
 2   sname   float64
dtypes: float64(3)
memory usage: 67.5 MB


In [332]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.000000         89
2.000000      56364
1.000000    2051152
0.691077          3
0.545380          3
0.503043          3
0.351428         13
0.115821          6
0.000000     162800
dtype: int64

In [333]:
potential_matches = features[(features.sum(axis=1) > 1.85) & (features.sname>0)].reset_index()


In [334]:

potential_matches['wikidata_id'] = wikidata.iloc[potential_matches['level_0']]['wikidata_id'].values
potential_matches['wname'] = wikidata.iloc[potential_matches['level_0']]['name'].values
potential_matches['wnat'] = wikidata.iloc[potential_matches['level_0']]['naturalidade'].values

potential_matches['fauc_id'] = fauc.iloc[potential_matches['level_1']]['fauc_id'].values
potential_matches['fname'] = fauc.iloc[potential_matches['level_1']]['name'].values
potential_matches['fnat'] = fauc.iloc[potential_matches['level_1']]['naturalidade'].values

potential_matches['score'] = potential_matches['scoord'] + potential_matches['snat'] + potential_matches['sname']

potential_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   level_0      400 non-null    int64  
 1   level_1      400 non-null    int64  
 2   scoord       400 non-null    float64
 3   snat         400 non-null    float64
 4   sname        400 non-null    float64
 5   wikidata_id  400 non-null    object 
 6   wname        400 non-null    object 
 7   wnat         400 non-null    object 
 8   fauc_id      400 non-null    object 
 9   fname        400 non-null    object 
 10  fnat         400 non-null    object 
 11  score        400 non-null    float64
dtypes: float64(4), int64(2), object(6)
memory usage: 37.6+ KB


In [335]:

potential_matches.sort_values(['wikidata_id','score','fauc_id'], ascending=[True,False,True], inplace=True)
potential_matches.head()

,level_0,level_1,scoord,snat,sname,wikidata_id,wname,wnat,fauc_id,fname,fnat,score
240,566,99498,0.0,1.0,1.0,http://www.wikidata.org/entity/Q10278167,Eugénio Vaz Pacheco do Canto e Castro,Ponta Delgada,132517,Eugénio Vaz Pacheco do Canto e Castro,Ponta Delgada,2.0
242,1066,99498,0.0,1.0,1.0,http://www.wikidata.org/entity/Q10278167,Eugénio Vaz Pacheco do Canto e Castro,Ponta Delgada,132517,Eugénio Vaz Pacheco do Canto e Castro,Ponta Delgada,2.0
367,585,9785,1.0,1.0,1.0,http://www.wikidata.org/entity/Q10281435,Fernando Martins de Mascarenhas,Montemor-o-Novo,170747,Fernão Martins Mascarenhas,Montemor-o-Novo,3.0
369,1082,9785,1.0,1.0,1.0,http://www.wikidata.org/entity/Q10281435,Fernando Martins de Mascarenhas,Montemor-o-Novo,170747,Fernão Martins Mascarenhas,Montemor-o-Novo,3.0
368,585,9901,1.0,1.0,1.0,http://www.wikidata.org/entity/Q10281435,Fernando Martins de Mascarenhas,Montemor-o-Novo,170751,Fernão Martins Mascarenhas,Montemor-o-Novo,3.0


In [336]:
from datetime import datetime, date



min_age = 14  #  minimum age at entry
max_age = 35  # max age at entry

potential_matches['date_match'] = None

for indx,wndx,fndx,scoord,snat,sname,wikidata_id,wname,wnat,fauc_id,fname,fnat,score,date_match in potential_matches.itertuples():
    birth_date = wikidata.loc[wndx,'birth_date']
    uc_entry = fauc.loc[fndx,'uc-entrada']
    if birth_date is not None:
        bdv = datetime.strptime(birth_date,'%Y-%m-%d')
        entry_year_from = date(bdv.year + min_age,1,1)
        entry_year_until = date(bdv.year + max_age,1,1)
        from_str = datetime.strftime(entry_year_from,'%Y-%m-%d')
        until_str = datetime.strftime(entry_year_until,'%Y-%m-%d')
        dates_in = (from_str, until_str)
    if uc_entry > from_str and uc_entry < until_str:
        potential_matches.loc[indx,'date_match'] = 1

potential_matches.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 240 to 17
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   level_0      400 non-null    int64  
 1   level_1      400 non-null    int64  
 2   scoord       400 non-null    float64
 3   snat         400 non-null    float64
 4   sname        400 non-null    float64
 5   wikidata_id  400 non-null    object 
 6   wname        400 non-null    object 
 7   wnat         400 non-null    object 
 8   fauc_id      400 non-null    object 
 9   fname        400 non-null    object 
 10  fnat         400 non-null    object 
 11  score        400 non-null    float64
 12  date_match   303 non-null    object 
dtypes: float64(4), int64(2), object(7)
memory usage: 59.9+ KB


In [337]:
matches_dated = potential_matches.loc[potential_matches['date_match'].notnull()].drop_duplicates(['wikidata_id','fauc_id','score'])
matches_dated[['wikidata_id','wname','score','fauc_id','fname','sname','wnat','fnat','snat','scoord']].sort_values(['wikidata_id','score','fauc_id'], ascending=[True,False,True]).head(20)

,wikidata_id,wname,score,fauc_id,fname,sname,wnat,fnat,snat,scoord
240,http://www.wikidata.org/entity/Q10278167,Eugénio Vaz Pacheco do Canto e Castro,2.0,132517,Eugénio Vaz Pacheco do Canto e Castro,1.0,Ponta Delgada,Ponta Delgada,1.0,0.0
367,http://www.wikidata.org/entity/Q10281435,Fernando Martins de Mascarenhas,3.0,170747,Fernão Martins Mascarenhas,1.0,Montemor-o-Novo,Montemor-o-Novo,1.0,1.0
368,http://www.wikidata.org/entity/Q10281435,Fernando Martins de Mascarenhas,3.0,170751,Fernão Martins Mascarenhas,1.0,Montemor-o-Novo,Montemor-o-Novo,1.0,1.0
104,http://www.wikidata.org/entity/Q10282299,Filipe de Sousa e Holstein,2.0,190218,Filipe de Sousa Holstein,1.0,Lisboa,Lisboa,1.0,0.0
231,http://www.wikidata.org/entity/Q10284873,Francisco José da Costa e Amaral,3.0,145473,Francisco José da Costa e Amaral,1.0,Portalegre,Portalegre,1.0,1.0
29,http://www.wikidata.org/entity/Q10287041,Félix José da Costa Sotto Mayor,2.0,242857,Felix José da Costa Soutomaior,1.0,Angra do Heroísmo,"Angra do Heroísmo, Açores",0.0,1.0
30,http://www.wikidata.org/entity/Q10292299,Nicolau Moniz de Bettencourt,2.0,280794,Nicolau Moniz de Bettencourt,1.0,Angra do Heroísmo,"Angra do Heroísmo, Açores",0.0,1.0
388,http://www.wikidata.org/entity/Q10294753,Henrique Ferreira de Oliveira Brás,2.0,188146,Henrique Ferreira de Oliveira Brás,1.0,Angra do Heroísmo,"Angra de Heroísmo, Açores",0.0,1.0
31,http://www.wikidata.org/entity/Q10307298,Joaquim Maria da Silva,2.0,227536,Joaquim Maria da Silva,1.0,Angra do Heroísmo,"Angra do Heroísmo, Açores",0.0,1.0
371,http://www.wikidata.org/entity/Q10307317,Joaquim Mendes dos Remédios,3.0,137672,Joaquim Mendes dos Remédios,1.0,Nisa,Nisa,1.0,1.0


In [338]:
print("Number of wikidata records matched:",len(matches_dated['wikidata_id'].unique()))
print("Number of fauc records matched:",len(matches_dated['fauc_id'].unique()))

Number of wikidata records matched: 122
Number of fauc records matched: 125


In [339]:
nat_analysis = potential_matches[['level_0','level_1','wnat','fnat','snat','scoord']].copy()
nat_analysis['wlat'] = wikidata.iloc[nat_analysis['level_0']]['latitude'].values
nat_analysis['wlong'] = wikidata.iloc[nat_analysis['level_0']]['longitude'].values

nat_analysis['flat'] = fauc.iloc[nat_analysis['level_1']]['latitude'].values
nat_analysis['flong'] = fauc.iloc[nat_analysis['level_1']]['longitude'].values

In [340]:
nat_analysis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 240 to 17
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   level_0  400 non-null    int64  
 1   level_1  400 non-null    int64  
 2   wnat     400 non-null    object 
 3   fnat     400 non-null    object 
 4   snat     400 non-null    float64
 5   scoord   400 non-null    float64
 6   wlat     400 non-null    float64
 7   wlong    400 non-null    float64
 8   flat     400 non-null    float64
 9   flong    400 non-null    float64
dtypes: float64(6), int64(2), object(2)
memory usage: 50.5+ KB


In [342]:
nat_analysis[(nat_analysis.snat > 0) & (nat_analysis.scoord == 0)].head(50)

,level_0,level_1,wnat,fnat,snat,scoord,wlat,wlong,flat,flong
240,566,99498,Ponta Delgada,Ponta Delgada,1.0,0.0,37.740000,-25.670000,32.828207,-16.987449
242,1066,99498,Ponta Delgada,Ponta Delgada,1.0,0.0,37.740000,-25.670000,32.828207,-16.987449
104,582,96077,Lisboa,Lisboa,1.0,0.0,38.708042,-9.139016,0.000000,0.000000
151,1079,96077,Lisboa,Lisboa,1.0,0.0,38.708042,-9.139016,0.000000,0.000000
353,613,647,Lagos,Lagos,1.0,0.0,37.100000,-8.666667,37.000816,-25.075449
354,620,647,Lagos,Lagos,1.0,0.0,37.100000,-8.666667,37.000816,-25.075449
355,623,647,Lagos,Lagos,1.0,0.0,37.100000,-8.666667,37.000816,-25.075449
356,629,647,Lagos,Lagos,1.0,0.0,37.100000,-8.666667,37.000816,-25.075449
357,636,647,Lagos,Lagos,1.0,0.0,37.100000,-8.666667,37.000816,-25.075449
358,1099,647,Lagos,Lagos,1.0,0.0,37.100000,-8.666667,37.000816,-25.075449


In [262]:

for indx,wndx,fndx,coord,nat,name,wikidata_id,fauc_id,score,date_match in potential_matches[potential_matches.date_match.notnul()].itertuples():
    wndx = wikidata['wikidata_id'] == w
    wname = wikidata.loc[wndx,'name']
    wnaturalidade = wikidata.loc[wndx,'naturalidade']
    wbirth_date = wikidata.loc[wndx,'birth_date']
    fauc_nome = fauc.loc[f,'name']
    fauc_naturalidade = fauc.loc[f,'naturalidade']
    fauc_uc_entrada = fauc.loc[f,'uc-entrada']
    print(f"{w}: {wname.iloc[0]}, {wnaturalidade.iloc[0]}, {wbirth_date.iloc[0]} -> {f}: {fauc_nome}, {fauc_naturalidade}, {fauc_uc_entrada}")


KeyError: '250574'

#### Testar diferentes funções de comparação de nomes

---

#### Testing different functions for comparing names


In [1]:
!pip install jellyfish

In [2]:
import jellyfish
# Examples of current misses
name1 = u'António Osório Sarmento de Figueiredo, Jr.'
name2 = u'António Osório Sarmento de Figueiredo Júnior'
print(name1,name2)
print(jellyfish.levenshtein_distance(name1,name2))
print(jellyfish.jaro_similarity(name1,name2))
print(jellyfish.jaro_winkler_similarity(name1,name2))

print()
print(name1,name2)
name1 = 'Samoa Barros'
name2 = 'Samora Barros'
print(jellyfish.levenshtein_distance(name1,name2))
print(jellyfish.jaro_similarity(name1,name2))
print(jellyfish.jaro_winkler_similarity(name1,name2))

print()
print(name1,name2)
name1 = 'Frei Custódio Alves Serrão'
name2 = 'Custódio Alves Serrão'
print(jellyfish.levenshtein_distance(name1,name2))
print(jellyfish.jaro_similarity(name1,name2))
print(jellyfish.jaro_winkler_similarity(name1,name2))


António Osório Sarmento de Figueiredo, Jr. António Osório Sarmento de Figueiredo Júnior
6
0.9538239538239538
0.9722943722943723
1
0.9188034188034188
0.9512820512820512


In [14]:
import warnings
from datetime import datetime, date

from timelinknb.pandas import attribute_to_df, remove_particles
from timelinknb import get_db
from ucalumni.config import default_db
from IPython.display import display

db_spec = default_db
db = get_db(db_spec)

students['fauc_id'] = None

not_found = 0
match = 0
ambiguous = 0

students.sort_values(['name','wikidata','alias','birth_date'], inplace=True)

match_list = []
not_found_list = []
duplicate_list = []
ambiguous_list = []

for i,row in students.iterrows():
    
    uri = row['wikidata']

    if uri.strip() == 'http://www.wikidata.org/entity/Q9618745'.strip():
        pass

    name = row['name']
    if uri in match_list or uri in not_found_list:
        print(f"Skipping {uri} {name} already checked in this run")
        continue

    if uri in manual_match.keys():
        fauc_id = manual_match[uri]
        students.loc[students['wikidata']==uri,'fauc_id'] = fauc_id
        print(f"Manually matched {fauc_id}={uri}")
        match_list.append(uri)
        match = match + 1
        continue

    # fix specific name problems in wilkipedia
    if name == 'Samoa Barros':
        name = 'Samora Barros'
    elif name == 'Frei Custódio Alves Serrão':
        name = 'Custódio Alves Serrão'
    elif name == 'Levy Maria Jordão':
        name = 'Levi Maria Jordão'
    elif name == 'Abel Maria Jordão de Paiva Manso':
        name = 'Abel Maria Jordão'
    elif name == 'Adriano António Canavarro Crispiniano da Fonseca':
        name = 'Adriano António Crispiniano da Fonseca'

    name_particles = remove_particles(name).split(" ")
    name_like = '%'.join(name_particles)
    name_like = '%'+name_like+'%'
    
    # break point zone.
    if name == 'Adriano António Canavarro Crispiniano da Fonseca':
        pass


    alias = row['alias']
    if alias is not None:
        alias_particles = remove_particles(alias).split(" ")
        alias_like = '%'.join(alias_particles)
        alias_like = '%'+alias_like+'%'

    if name is None and alias is None:
        warnings.warn(f"{uri} has null name and no alias, skipped.")
        continue
    
    naturalidade = row['naturalidade']

    birth_date = row['birth_date']
    if birth_date is not None:
        bdv = datetime.strptime(birth_date,'%Y-%m-%d')
        entry_year_from = date(bdv.year + min_age,1,1)
        entry_year_until = date(bdv.year + max_age,1,1)
        from_str = datetime.strftime(entry_year_from,'%Y-%m-%d')
        until_str = datetime.strftime(entry_year_until,'%Y-%m-%d')
        dates_in = (from_str, until_str)
    else:
        dates_in = ('1530-00-00','1910-00-00')


    if row['naturalidade'] is not None:

        aka = f"(aka {alias})" if alias is not None else ""
        print(f"Looking for {name} {aka} {naturalidade} {birth_date} {uri}", end=" ")
        
        # get students with similar names from similar places
        candidates = attribute_to_df('nome-geografico',the_value='%'+naturalidade+'%',name_like=name_like,dates_in=dates_in)
        
        if candidates is None and alias is not None:
            # Try again with alias
            candidates = attribute_to_df('nome-geografico',the_value='%'+naturalidade+'%',name_like=alias_like,dates_in=dates_in)

        if candidates is None:
            print("Not found")
            not_found = not_found + 1
            not_found_list.append(uri)
        else:
            # some candidates were found
            # if just one we take it
            if len(candidates.index) == 1:
                fauc_id = candidates.iloc[0].name
                match = match + 1
                students.loc[students['wikidata']==uri,'fauc_id'] = fauc_id
                print(f"Found {fauc_id}={uri}")
                match_list.append(uri)
            else:  # found more than one check if one name matches exactly
                hit = 0
                candidates_list = []
                for fid, row in candidates.iterrows():
                    if remove_particles(row['name']).lower() == remove_particles(name).lower():
                        hit = hit + 1
                        hit_id = fid
                        candidates_list.append(fid)
                if hit == 0 and alias is not None:
                    hit = 0
                    for fid, row in candidates.iterrows():
                        if remove_particles(row['name']).lower() == remove_particles(alias).lower():
                            hit = hit + 1
                            hit_id = fid      
                            candidates_list.append(fid)             
                if hit == 0:
                    print("Ambiguous: could not find match in candidates, check possible duplicates in FAUC")
                    display(candidates.head(10))
                    ambiguous = ambiguous + 1
                    ambiguous_list.append({'widata':uri,'name':name, 'fauc':list(candidates.index.unique())})
                    not_found_list.append(uri)
                elif hit > 0:
                    match = match + 1
                    students.loc[students['wikidata']==uri,'fauc_id'] = fauc_id
                    print(f"Found {fauc_id}={uri}")
                    match_list.append(uri)
                    if hit > 1 :
                        duplicate_list.append({'widata':uri,'name':name, 'fauc':candidates_list})
                        warnings.warn("Possible duplicates: "+", ".join(candidates_list))

print(f"Matched {match} ambiguous {ambiguous} not found {not_found}")



Looking for Aarão Ferreira de Lacerda  Vila Nova de Paiva 1863-10-03 http://www.wikidata.org/entity/Q9559855 Not found
Looking for Abel Maria Jordão  Coimbra 1801-03-03 http://www.wikidata.org/entity/Q106271718 Not found
Looking for Abílio Augusto da Fonseca Pinto  Coimbra 1830-05-27 http://www.wikidata.org/entity/Q23820937 Found 245300=http://www.wikidata.org/entity/Q23820937
Looking for Adolfo Ferreira Loureiro (aka Adolfo Ferreira de Loureiro) São Bartolomeu 1836-12-12 http://www.wikidata.org/entity/Q19607365 Not found
Looking for Adriano António Crispiniano da Fonseca  Marco de Canaveses 1884-11-08 http://www.wikidata.org/entity/Q16144296 Found 139360=http://www.wikidata.org/entity/Q16144296
Looking for Adriano Inácio Botelho  Angra do Heroísmo 1892-09-12 http://www.wikidata.org/entity/Q56072395 Found 186608=http://www.wikidata.org/entity/Q56072395
Looking for Afonso Costa (aka Afonso Augusto Costa) Seia 1871-03-06 http://www.wikidata.org/entity/Q383401 Found 198981=http://www.wiki

,name,sex,nome-geografico,nome-geografico.date,nome-geografico.obs
id,,,,,
236173,Fernando Diogo de Sousa da Silva e Menezes,m,Lisboa,1730-10-01,None
226251,Fernando Diogo de Sousa e Silva,m,Lisboa,1731-10-01,None


Looking for Filipe Folque (aka Filipe de Sousa Folque) Portalegre 1800-11-28 http://www.wikidata.org/entity/Q17280326 Found 139332=http://www.wikidata.org/entity/Q17280326
Looking for Filipe de Sousa e Holstein (aka Filipe de Sousa Holstein) Lisboa 1841-12-26 http://www.wikidata.org/entity/Q10282299 Found 190218=http://www.wikidata.org/entity/Q10282299
Looking for Filomeno da Câmara Melo Cabral  Lagoa 1844-07-18 http://www.wikidata.org/entity/Q10282363 Not found
Looking for Francisco Antonio Pereira da Costa  Lisboa 1809-10-11 http://www.wikidata.org/entity/Q55908387 Not found
Looking for Francisco António Fernandes da Silva Ferrão  Coimbra 1798-07-23 http://www.wikidata.org/entity/Q17280331 Not found
Looking for Francisco Caupers Machado de Faria e Maia  Ponta Delgada 1841-05-24 http://www.wikidata.org/entity/Q10284691 Not found
Looking for Francisco Correia de Mendonça  Lagos 1832-01-03 http://www.wikidata.org/entity/Q16917757 Found 234635=http://www.wikidata.org/entity/Q16917757
Loo

/var/folders/bc/41rz3hqn5335_y1tbvy1w2_w0000gp/T/ipykernel_37701/592778839.py:143: UserWarning: Possible duplicates: 186265, 131212
  warnings.warn("Possible duplicates: "+", ".join(candidates_list))


Not found
Looking for José Joaquim Carneiro de Campos (aka José Joaquim Carneiro de Santos) Salvador 1768-03-04 http://www.wikidata.org/entity/Q10309643 Not found
Skipping http://www.wikidata.org/entity/Q10309643 José Joaquim Carneiro de Campos already checked in this run
Skipping http://www.wikidata.org/entity/Q10309643 José Joaquim Carneiro de Campos already checked in this run
Skipping http://www.wikidata.org/entity/Q10309643 José Joaquim Carneiro de Campos already checked in this run
Skipping http://www.wikidata.org/entity/Q10309643 José Joaquim Carneiro de Campos already checked in this run
Looking for José Joaquim Justiniano Mascarenhas Castelo Branco  Rio de Janeiro 1731-08-23 http://www.wikidata.org/entity/Q64734418 Not found
Looking for José Joaquim Rodrigues de Bastos  Moitedo 1777-11-08 http://www.wikidata.org/entity/Q21483097 Not found
Looking for José Joaquim da Cunha Azeredo Coutinho (aka Azeredo Coutinho) Rio de Janeiro 1742-09-08 http://www.wikidata.org/entity/Q4290353 

,name,sex,nome-geografico,nome-geografico.date,nome-geografico.obs
id,,,,,
198631,José Luciano de Castro Pereira Corte Real,m,Oliveirinha,1849-10-04,None
292664,José Luciano de Castro Pereira Corte Real,m,Oliveirinha,1853-07-06,"Oliveirinha, Aveiro"


Skipping http://www.wikidata.org/entity/Q321218 José Luciano de Castro already checked in this run
Skipping http://www.wikidata.org/entity/Q321218 José Luciano de Castro already checked in this run
Skipping http://www.wikidata.org/entity/Q321218 José Luciano de Castro already checked in this run
Looking for José Luís Alves Feijó  Freixo de Espada à Cinta 1816-01-08 http://www.wikidata.org/entity/Q64697672 Not found
Looking for José Luís Soares de Barbosa  Setúbal 1728-09-29 http://www.wikidata.org/entity/Q105235831 Found 150836=http://www.wikidata.org/entity/Q105235831
Looking for José Luís de Vasconcelos e Sousa  Rio de Janeiro 1740-07-09 http://www.wikidata.org/entity/Q107328945 Not found
Looking for José Machado Serpa (aka José Machado de Serpa) Prainha 1864-03-09 http://www.wikidata.org/entity/Q10309855 Found 223548=http://www.wikidata.org/entity/Q10309855
Looking for José Malheiro Reimão (aka José Malheiro Reimão Teles de Meneses e Sá) Viana do Castelo 1860-07-03 http://www.wikida

,name,sex,nome-geografico,nome-geografico.date,nome-geografico.obs
id,,,,,
172342,Arnaldo Mendes Norton de Matos,m,Ponte de Lima,1881-10-03,None
250289,Tomás Mendes Norton de Matos Prego,m,Ponte de Lima,1889-10-02,None
172343,Tomás Mendes Norton de Matos,m,Ponte de Lima,1891-06-25,None


Looking for Nuno Eugénio Lóssio e Seiblitz (aka Nuno Eugénio de Lossio) Recife 1782-10-01 http://www.wikidata.org/entity/Q10338177 Not found
Skipping http://www.wikidata.org/entity/Q10338177 Nuno Eugénio Lóssio e Seiblitz already checked in this run
Skipping http://www.wikidata.org/entity/Q10338177 Nuno Eugénio Lóssio e Seiblitz already checked in this run
Looking for Nuno da Cunha e Ataíde (aka Nuno da cunha e ataíde) Lisboa 1664-12-08 http://www.wikidata.org/entity/Q3346439 Found 148662=http://www.wikidata.org/entity/Q3346439
Looking for Pedro António Correia Garção  Lisboa 1724-04-29 http://www.wikidata.org/entity/Q1515561 Found 177304=http://www.wikidata.org/entity/Q1515561
Looking for Pedro Augusto Ferreira (aka Pedro augusto ferreira) Lamego 1833-11-14 http://www.wikidata.org/entity/Q10347323 Not found
Looking for Pedro José Constâncio  Lisboa 1781-11-03 http://www.wikidata.org/entity/Q15631832 Found 195967=http://www.wikidata.org/entity/Q15631832
Looking for Pedro Paulo de Figue

In [16]:
students.to_csv('../inferences/wikidata/students_wikidata_matched.csv',sep=',')

#### Processar os nomes não ligados no primeiro passo

No segundo passo usamos simplesmente o nome e as data para recolher candidatos.
Depois testamos naturalidade se disponível.

---

#### Processing names not linked in the first pass

In the second pass we collect candidates by name and dates.
We test for place of birth onyl if available.


In [17]:
unlinked = students[students['fauc_id'].isnull()]
unlinked[['name','alias','naturalidade','birth_date','wikidata']].head().fillna("")

,name,alias,naturalidade,birth_date,wikidata
41,Aarão Ferreira de Lacerda,,Vila Nova de Paiva,1863-10-03,http://www.wikidata.org/entity/Q9559855
673,Abel Maria Jordão de Paiva Manso,,Coimbra,1801-03-03,http://www.wikidata.org/entity/Q106271718
9,Abílio Beça,,,1856-08-20,http://www.wikidata.org/entity/Q9561614
580,Adolfo Ferreira Loureiro,Adolfo Ferreira de Loureiro,São Bartolomeu,1836-12-12,http://www.wikidata.org/entity/Q19607365
253,Afonso Mendes,,Santo Aleixo da Restauração,1579-06-28,http://www.wikidata.org/entity/Q4689383


In [18]:
from datetime import datetime,date
import pandas as pd
from timelinknb.pandas import name_to_df,attribute_to_df, remove_particles

match_list = []
not_found_list = []
for i, row in unlinked.iterrows():
    # display(unlinked.loc[[i]][['name','alias','naturalidade','birth_date','wikidata']])
    name = row['name']
    alias = row['alias']
    uri = row['wikidata']
    naturalidade = row['naturalidade']
    fauc_id_previous = row['fauc_id']
    
    birth_date = row['birth_date']
    if birth_date is not None:
        bdv = datetime.strptime(birth_date,'%Y-%m-%d')
        entry_year_from = date(bdv.year + min_age,1,1)
        entry_year_until = date(bdv.year + max_age,1,1)
        from_str = datetime.strftime(entry_year_from,'%Y-%m-%d')
        until_str = datetime.strftime(entry_year_until,'%Y-%m-%d')
        dates_in = (from_str, until_str)
    else:
        dates_in = ('1530-00-00','1910-00-00')

    print(f"Looking for {name} ({alias}), {naturalidade} {birth_date} {uri}", end=" ")
    if fauc_id_previous is not None:
        print(f"Found previously in this run {fauc_id_previous}={uri}")
        continue
    if uri in match_list or uri in not_found_list:
        print(f"Skipping {uri} {name} already checked in this run")
        continue

    
    if uri == 'http://www.wikidata.org/entity/Q10281435':
        pass
    if alias == 'António Vasco de Melo':
        pass

    candidates = name_to_df(name, similar=True)
    if candidates is None and alias is not None:
        candidates = name_to_df(alias, similar=True)
    if candidates is None:
        print(f"Not found in FAUC.")
    else:
        candidates_attributes = attribute_to_df('uc-entrada',more_cols=['uc-saida','naturalidade'],
                                                filter_by=list(candidates.index.unique()),
                                                dates_in=dates_in)
        if candidates_attributes is not None:
            if len(candidates_attributes.index.unique()) == 1:  # we have only one candidates we match
                # display(candidates_attributes)
                fauc_id = candidates_attributes.index.unique()[0]
                students.loc[students['wikidata']==uri,'fauc_id'] = fauc_id
                print(f"Found: {fauc_id}.")
                match_list.append(uri)
            else:
                #  More than one candidate. Check for exact match on name or alias
                candidates_attributes['name'] = candidates_attributes['name'].apply(remove_particles)
                if alias is None:
                    s = ''
                else:
                    s = alias
                match = (candidates_attributes['name'] == remove_particles(s)) \
                    | (candidates_attributes['name'] == remove_particles(name))
                if len(candidates_attributes[match].index.unique()) == 1:
                    # display(candidates_attributes)
                    fauc_id = candidates_attributes.index.unique()[0] # should detect already linked, possible duplicate
                    students.loc[students['wikidata']==uri,'fauc_id'] = fauc_id
                    print(f"Found {fauc_id}.")
                    match_list.append(uri)
                else:
                    # we try to see if there is a matching place of birth
                    match = (candidates_attributes['naturalidade'] == naturalidade)
                    if len(candidates_attributes[match].index.unique()) == 1:
                        fauc_id = candidates_attributes.index.unique()[0] # should detect already linked, possible duplicate
                        print(f"Found {fauc_id}.")
                        # display(candidates_attributes)
                        students.loc[students['wikidata']==uri,'fauc_id'] = fauc_id   
                        match_list.append(uri) 
                    else:           
                        print("Multiple candidates, no exact name or alias match")
                        display(candidates_attributes[['name','uc-entrada','naturalidade']])
                        ambiguous = ambiguous + 1
                        ambiguous_list.append({'widata':uri,'name':name, 'fauc':list(candidates_attributes.index.unique())})

                
        else:  # check if we have a cross reference
            candidates_attributes = attribute_to_df('uc-entrada', the_value='0000-00-00',more_cols=['nome','naturalidade'],
                                                filter_by=list(candidates.index.unique()))
            if candidates_attributes is not None:
                print("Name exists in FAUC but is a cross reference (vide)", end=" ")
                candidates_attributes['name'] = candidates_attributes['name'].apply(remove_particles)
                if alias is None:
                    s = ''
                else:
                    s = alias
                match = (candidates_attributes['name'] == remove_particles(s)) \
                    | (candidates_attributes['name'] == remove_particles(name))
                if len(candidates_attributes[match].index.unique()) == 1:
                    # display(candidates_attributes)
                    fauc_id = candidates_attributes.index.unique()[0] # should detect already linked, possible duplicate
                    students.loc[students['wikidata']==uri,'fauc_id'] = fauc_id
                    print(f"Found {fauc_id}.")
                    match_list.append(uri) 
                elif len(candidates_attributes[match].index.unique()) > 1:
                    print("Multiple candidates, no exact name or alias match")
                    display(candidates_attributes[['name','uc-entrada','naturalidade']])
                    ambiguous = ambiguous + 1
                    ambiguous_list.append({'widata':uri,'name':name, 'fauc':list(candidates_attributes.index.unique())})      
                else:
                    print(f"Not found.")
            else:
                print(f"Not found. \n     Name exists in FAUC but not as a contemporary student, could be a relative.")
                candidates_attributes = attribute_to_df('uc-entrada',more_cols=['naturalidade','uc-saida'],
                                                         filter_by=list(candidates.index.unique()))
                if candidates_attributes is None:
                    display(candidates)
                else:
                    display(candidates_attributes[['name','uc-entrada','naturalidade']])

Looking for Aarão Ferreira de Lacerda (None), Vila Nova de Paiva 1863-10-03 http://www.wikidata.org/entity/Q9559855 Found: 265860.
Looking for Abel Maria Jordão de Paiva Manso (None), Coimbra 1801-03-03 http://www.wikidata.org/entity/Q106271718 Not found. 
     Name exists in FAUC but not as a contemporary student, could be a relative.


,name,sex,obs
id,,,
193376-pai,Abel Maria Jordão Paiva Manso,m,None


Looking for Abílio Beça (None), None 1856-08-20 http://www.wikidata.org/entity/Q9561614 Found: 185067.
Looking for Adolfo Ferreira Loureiro (Adolfo Ferreira de Loureiro), São Bartolomeu 1836-12-12 http://www.wikidata.org/entity/Q19607365 Found: 253929.
Looking for Afonso Mendes (None), Santo Aleixo da Restauração 1579-06-28 http://www.wikidata.org/entity/Q4689383 Found: 204790.
Looking for Aires Jácome Correia (None), Lisboa 1882-08-09 http://www.wikidata.org/entity/Q16144891 Not found in FAUC.
Looking for Alberto Teles de Utra Machado (None), Angra do Heroísmo 1840-01-24 http://www.wikidata.org/entity/Q16487343 Found: 181627.
Looking for Albino dos Reis (Albino Soares Pinto dos Reis), Oliveira de Azeméis 1888-09-30 http://www.wikidata.org/entity/Q9594728 Found 316083.
Looking for Albino dos Reis (Albino Soares Pinto dos Reis Júnior), Oliveira de Azeméis 1888-09-30 http://www.wikidata.org/entity/Q9594728 Skipping http://www.wikidata.org/entity/Q9594728 Albino dos Reis already checked i

,name,uc-entrada,naturalidade
id,,,
174441,António Ferreira de Lemos Malheiro Vasconcelos...,1787-00-00,Resende
174551,António de Vasconcelos Pereira de Melo,1840-10-09,Santa Cristina de Figueiró
235520,António Vasco de Melo Silva César e Menezes,1893-10-09,Lisboa


Looking for António Maria Vasco de Melo Silva César e Meneses (António de Melo César de Meneses), Lisboa 1851-11-13 http://www.wikidata.org/entity/Q9619744 Not found in FAUC.
Looking for António Maria Vasco de Melo Silva César e Meneses (António de Melo César e Meneses), Lisboa 1851-11-13 http://www.wikidata.org/entity/Q9619744 Not found in FAUC.
Looking for António Maria Vasco de Melo Silva César e Meneses (António de melo césar de meneses), Lisboa 1851-11-13 http://www.wikidata.org/entity/Q9619744 Not found in FAUC.
Looking for António Maria Vasco de Melo Silva César e Meneses (António de melo césar e meneses), Lisboa 1851-11-13 http://www.wikidata.org/entity/Q9619744 Not found in FAUC.
Looking for António Maria Vasco de Melo Silva César e Meneses (António maria vasco de mello silva césar e menezes), Lisboa 1851-11-13 http://www.wikidata.org/entity/Q9619744 Not found in FAUC.
Looking for António Maria Vasco de Melo Silva César e Meneses (António maria vasco de melo césar e meneses), 

...

Looking for Pedro de Sousa Holstein, Duque de Palmela (Pedro de Souza Holstein), Turim 1781-05-08 http://www.wikidata.org/entity/Q982358 Not found in FAUC.
Looking for Q15486217 (None), None 1780-05-10 http://www.wikidata.org/entity/Q15486217 Not found in FAUC.
Looking for Q1771793 (None), Waterford 1588-10-16 http://www.wikidata.org/entity/Q1771793 Not found in FAUC.
Looking for Q20980122 (None), Lisboa 1587-09-26 http://www.wikidata.org/entity/Q20980122 Not found in FAUC.
Looking for Q4777580 (None), None 1859-09-21 http://www.wikidata.org/entity/Q4777580 Not found in FAUC.
Looking for Q6844431 (None), None 1831-08-22 http://www.wikidata.org/entity/Q6844431 Not found in FAUC.
Looking for Q7442698 (None), Seia 1842-12-17 http://www.wikidata.org/entity/Q7442698 Not found in FAUC.
Looking for Rafael Duque (Rafael da Silva Neves Duque), Torres Novas 1893-02-03 http://www.wikidata.org/entity/Q10357034 Not found in FAUC.
Looking for Raul Leal (None), Lisboa 1886-09-01 http://www.wikidata.o

,name,uc-entrada,naturalidade
id,,,
248766,Francisco de Sá Miranda,1677-10-01,Pedrogão


Looking for Teotónio Simão Paim de Ornelas Bruges (Teotónio Paim de Ornelas Bruges), Angra do Heroísmo 1841-09-22 http://www.wikidata.org/entity/Q10380409 Not found in FAUC.
Looking for Tomás António Gonzaga (Thomaz Antonio Gonzaga), Miragaia 1744-08-11 http://www.wikidata.org/entity/Q1334602 Found: 179856.
Looking for Tomás António Gonzaga (Tomas António Gonzaga), Miragaia 1744-08-11 http://www.wikidata.org/entity/Q1334602 Skipping http://www.wikidata.org/entity/Q1334602 Tomás António Gonzaga already checked in this run
Looking for Tomás António Gonzaga (Tomás Gonzaga), Miragaia 1744-08-11 http://www.wikidata.org/entity/Q1334602 Skipping http://www.wikidata.org/entity/Q1334602 Tomás António Gonzaga already checked in this run
Looking for Tomás António Gonzaga (Tomáz António Gonzaga), Miragaia 1744-08-11 http://www.wikidata.org/entity/Q1334602 Skipping http://www.wikidata.org/entity/Q1334602 Tomás António Gonzaga already checked in this run
Looking for Tomás Cabreira (Tomás António da 

,name,uc-entrada,naturalidade
id,,,
144221,Tomás Almeida porcionista São Paulo),1693-11-25,NaN
245209,Tomás Almeida Pinho,1699-02-17,Aveiro


Looking for Tomás de Sousa e Holstein Beck (None), Lisboa 1839-12-31 http://www.wikidata.org/entity/Q10383143 Not found in FAUC.
Looking for Toríbio de Mongrovejo (None), Mayorga 1538-11-26 http://www.wikidata.org/entity/Q467993 Not found in FAUC.
Looking for Venceslau de Lima (Venceslau de Sousa Pereira de Lima), Porto 1858-11-15 http://www.wikidata.org/entity/Q1392611 Not found in FAUC.
Looking for Venceslau de Lima (Wenceslau de Lima), Porto 1858-11-15 http://www.wikidata.org/entity/Q1392611 Not found in FAUC.
Looking for Venceslau de Lima (Wenceslau de Sousa Pereira de Lima), Porto 1858-11-15 http://www.wikidata.org/entity/Q1392611 Not found in FAUC.
Looking for Vicente Ferrer Neto de Paiva (Vicente Ferrer Neto Paiva), Lousã 1798-06-27 http://www.wikidata.org/entity/Q10390244 Found: 222498.
Looking for Vicente Ferrer Neto de Paiva (Vicente Ferrer de Neto Paiva), Lousã 1798-06-27 http://www.wikidata.org/entity/Q10390244 Skipping http://www.wikidata.org/entity/Q10390244 Vicente Ferre

In [11]:
students.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 691 entries, 41 to 193
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   wikidata      691 non-null    object
 1   name          691 non-null    object
 2   alias         464 non-null    object
 3   bnp_id        458 non-null    object
 4   naturalidade  654 non-null    object
 5   placeID       654 non-null    object
 6   longitude     652 non-null    object
 7   latitude      652 non-null    object
 8   birth_date    691 non-null    object
 9   fauc_id       471 non-null    object
dtypes: object(10)
memory usage: 59.4+ KB


In [12]:
students.to_csv('../inferences/wikidata/students_wikidata_matched.csv',sep=',')

In [22]:
unlinked = students[students['fauc_id'].isnull()]
print("Wikidata entries not matched:",len(list(unlinked['wikidata'].unique())))
print(unlinked.info())
unlinked[['name','alias','naturalidade','birth_date','wikidata']].head(30).fillna("")[15:]

Wikidata entries not matched: 146
<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 673 to 293
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   wikidata      220 non-null    object
 1   name          220 non-null    object
 2   alias         134 non-null    object
 3   bnp_id        125 non-null    object
 4   naturalidade  203 non-null    object
 5   placeID       203 non-null    object
 6   longitude     203 non-null    object
 7   latitude      203 non-null    object
 8   birth_date    220 non-null    object
 9   fauc_id       0 non-null      object
dtypes: object(10)
memory usage: 18.9+ KB
None


,name,alias,naturalidade,birth_date,wikidata
82,António Maria Vasco de Melo Silva César e Meneses,António maria vasco de melo césar e meneses,Lisboa,1851-11-13,http://www.wikidata.org/entity/Q9619744
47,António Maria Vasco de Melo Silva César e Meneses,António vasco de melo,Lisboa,1851-11-13,http://www.wikidata.org/entity/Q9619744
157,António Maria da Silva,António maria da silva,Lisboa,1872-05-26,http://www.wikidata.org/entity/Q611175
632,António Menano,,Fornos de Algodres,1895-05-05,http://www.wikidata.org/entity/Q16492076
665,António Mendo Caldeira de Castel-Branco Cota F...,,Portalegre,1848-10-09,http://www.wikidata.org/entity/Q16492080
226,"António Osório Sarmento de Figueiredo, Jr.",,Cumieira,1855-03-13,http://www.wikidata.org/entity/Q4777591
197,António Teles Peixoto Guterres Palhinha,,Horta,1813-10-11,http://www.wikidata.org/entity/Q63981883
16,António da Trindade de Vasconcelos Pereira de ...,,Amarante,1812-08-28,http://www.wikidata.org/entity/Q9620375
118,Antônio Carlos Ribeiro de Andrada Machado e Silva,,Santos,1773-11-01,http://www.wikidata.org/entity/Q742361
674,Antônio José Machado,,Fortaleza,1809-10-14,http://www.wikidata.org/entity/Q16491822


Current misses:
* Abel Maria Jordão de Paiva Manso		Coimbra	1801-03-03	http://www.wikidata.org/entity/Q106271718
  * FAUC has him as father: https://linode.timelink-mhk.net/mhk/ucalumni/id/193376-pai 
  * And him as student: https://linode.timelink-mhk.net/mhk/ucalumni/id/193254 but gives place name as "Buarcos". Father name match genealogical information at: 
        https://gw.geneanet.org/aqf?lang=fr&n=manso&oc=0&p=abel+maria+jordao+de+paiva 
  * See https://github.com/joaquimrcarvalho/fauc1537-1919/issues/13

* FIXED Adriano António Canavarro Crispiniano da Fonseca		Marco de Canaveses	1884-11-08	http://www.wikidata.org/entity/Q16144296
  * exists as https://linode.timelink-mhk.net/mhk/ucalumni/id/139360 Fails because the FAUC does not have the Canavarro. Fixed with
    name override in the first pass.
    
* Aires Jácome Correia		Lisboa	1882-08-09	http://www.wikidata.org/entity/Q16144891
  * Not found: a probable ancestor exists:   https://linode.timelink-mhk.net/mhk/ucalumni/id/148381
  * True miss.

* Alfredo da Silva Sampaio		Angra do Heroísmo	1872-09-19	http://www.wikidata.org/entity/Q9603234
  * not found

* FIXED Almeida Garrett	Almeida Garret	Porto	1799-02-04	http://www.wikidata.org/entity/Q316806
  * FAUC Name João Batista da Silva Leitão id: 203942  Solved by manual linking

* Almiro Maia de Loureiro		Lisboa	1897-09-21	http://www.wikidata.org/entity/Q23777977
  * Not found Not clear from bio that he was a regular student.

* FIXED Amadeu de Castro e Sola	Amadeu Teles da Silva de Afonseca Mesquita de ...	Braga	1874-08-19	http://www.wikidata.org/entity/Q20045590
   * Amadeu da Costa Pereira e Sola id: 233633. Solved in manual matches.

* FIXED António Bernardo da Costa Cabral, Marquês de T...		Fornos de Algodres	1803-05-09	http://www.wikidata.org/entity/Q130870
  * Name in Wikidata contains title. Fixed with manual match.
    
* FIXED António Borges Coutinho de Medeiros de Sousa Dias da Câmara  https://www.wikidata.org/wiki/Q9618745
    * https://linode.timelink-mhk.net/mhk/ucalumni/id/235243 António Borges de Merdeiros Dias da Câmara e Sousa, fixed with manual
      match.
    
* Antonio Cordeses		Olot	1518-08-09	http://www.wikidata.org/entity/Q27786526 
    * Bio says professor and rector, not student. Likely rector of the Jesuit College, not the University. Bad wikidata entry.

* FIXED António Dinis da Graça Vieira			1824-07-26	http://www.wikidata.org/entity/Q18707730
    * corresponde a 216781 Mssies because name in FAUC is António Dinis Vieira

* FIXED MANUAL Amadeu de Castro e Sola	Amadeu Teles da Silva de Afonseca Mesquita de ...	Braga	1874-08-19	http://www.wikidata.org/entity/Q20045590
  
* 	António Maria Vasco de Melo Silva César e Meneses	António Vasco de Melo	Lisboa	1851-11-13	http://www.wikidata.org/entity/Q9619744 
    * Existe um homónimo que entra em 1893 FAUC: 235520. 
    *  Wikipedia https://pt.wikipedia.org/wiki/António_Maria_Vasco_de_Melo_Silva_César_e_Meneses: "António Vasco de Melo, nome que utilizou enquanto estudante e com o qual assinou muitas das suas obras literárias, era bacharel formado em Direito pela Universidade de Coimbra, onde terminou o seu curso em 1877".
    * Real Miss.

* António Ginestal Machado	Ginestal machado	Almeida	1874-05-03	http://www.wikidata.org/entity/Q357489
    * Parece ser um true miss.

* António Luís Ferreira Carneiro de Vasconcelos Teixeira Girão		1823-07-13	http://www.wikidata.org/entity/Q16491923
  * Miss

* 	António Maria Vasco de Melo Silva César e Meneses Lisboa	1851-11-13	http://www.wikidata.org/entity/Q9619744
  * https://linode.timelink-mhk.net/mhk/ucalumni/id/235520 possível filho ou erro de data
  * Miss

* António Menano		Fornos de Algodres	1895-05-05	http://www.wikidata.org/entity/Q16492076
    * Entra em 1915 está fora de scope. Temos o irmão https://pesquisa.auc.uc.pt/details?id=204362

* FIXED António Mendo Caldeira de Castel-Branco Cota Falcão (None), Portalegre 1848-10-09 http://www.wikidata.org/entity/Q16492080
    * FAUC António Mendes Caldeira Castelo Branco id: 189346 Misses because of name varation

* FIXED António Osório Sarmento de Figueiredo, Jr. (None), Cumieira 1855-03-13 http://www.wikidata.org/entity/Q4777591 
  * António Osório Sarmento de Figueiredo Júnior id: 138173 Misses because of name variations

* António Teles Peixoto Guterres Palhinha (None), Horta 1813-10-11 http://www.wikidata.org/entity/Q63981883
  * Not found, found father and son but not him.

* António da Trindade de Vasconcelos Pereira de Melo (None), Amarante 1812-08-28 http://www.wikidata.org/entity/Q9620375
  * António de Vasconcelos Pereira de Melo id: 174551

* Miguel José de Arriaga Brum da Silveira (None), Horta 1776-03-22 http://www.wikidata.org/entity/Q23771065 Not found in FAUC.
  * Miguel de Arriaga Brum da Silveira id: 230829

* FIXED Joaquim José Pereira da Silveira e Sousa (None), Velas 1796-02-17 http://www.wikidata.org/entity/Q10307234 
    * No FAUC é apenas Joaquim José Pereira da Silveira id: 230649 https://linode.timelink-mhk.net/mhk/ucalumni/id/230649



Previous misses:

* Aarão Ferreira de Lacerda  Vila Nova de Paiva 1863-10-03 http://www.wikidata.org/entity/Q9559855
  * FAUC place of birth Touro https://linode.timelink-mhk.net/mhk/ucalumni/id/265860 Wikidata: Vila Nova de Paiva

* Abílio Beça			1856-08-20	http://www.wikidata.org/entity/Q9561614
  * FAUC https://linode.timelink-mhk.net/mhk/ucalumni/id/185067 misses because no place of birth in wikidata.
* Looking António Augusto de Sousa 1883-12-10 http://www.wikidata.org/entity/Q23905297
  * Not found in FAUC  
* Alfredo Augusto de Soveral Martins Almacave 1869-06-07 http://www.wikidata.org/entity/Q1636522
  * Place name do not match. FAUC Lamego, Wikidata Almacave, which is parish
* Bernardo de Brito Almeida 1569-08-20 http://www.wikidata.org/entity/Q3638813
  * FAUC no birth place
* António Ginestal Machado Almeida 1874-05-03 http://www.wikidata.org/entity/Q357489
  * Not found in FAUC
* Levy Maria Jordão Lisboa 1831-01-09 http://www.wikidata.org/entity/Q94850304
  * Name in FAUC is Levi Maria Jordão
* António Maria Vasco de Melo Silva César e Meneses Lisboa 1851-11-13 http://www.wikidata.org/entity/Q9619744
  * FAUC name = António Vasco de Melo Silva César e Menezes
* Manuel Gonçalves Cerejeira Lousado 1888-11-29 http://www.wikidata.org/entity/Q966168
  * FAUC place of birth Lousada
* António de Sousa de Macedo Porto 1606-12-15 http://www.wikidata.org/entity/Q8202081
  * FAUC possible https://linode.timelink-mhk.net/mhk/ucalumni/id/183307 but place of birth does not match
* Pedro de Sousa Holstein, Duque de Palmela Turim 1781-05-08 http://www.wikidata.org/entity/Q982358
  * Homonym in https://linode.timelink-mhk.net/mhk/ucalumni/id/190226 but dates don't match also no one 
    born in Turim with similar name in FAUC
* António Pinto de Magalhães Aguiar Constance 1834-06-23 http://www.wikidata.org/entity/Q9620002
  * matches António Pinto de Magalhães Aguiar id: 141234 but place of birth does not match FAUC = Santa Eulália  
    de Constância
* Gregório de Matos, http://www.wikidata.org/entity/Q983565 corresponds to Gregório de Matos Guerra id: 180656 vide https://linode.timelink-mhk.net/mhk/ucalumni/id/171909
  * Misses because wikidata gives place of Birth as Salvador and FAUC gives Baía
'Anselmo José Braamcamp' 'Anselmo José Braancamp'

In [117]:
ginestal = attribute_to_df('grau.ano',the_value='Bacharel em Direito.1898%',name_like='António%', more_cols=['naturalidade'])
ginestal.sort_values('name')

,name,sex,grau.ano,grau.ano.date,grau.ano.obs,naturalidade,naturalidade.date,naturalidade.obs
id,,,,,,,,
179915,António Augusto Mendes de Gouveia,m,Bacharel em Direito.1898,1898-06-16,4.º e grau de Bacharel 16.06.1898 Aprovado Nem...,Sandomil,1894-10-02,None
247823,António Augusto de Miranda,m,Bacharel em Direito.1898,1898-06-20,4.º e grau de Bacharel 20.06.1898 Aprovado Sim...,Gestaçô,1894-10-01,None
310762,António Ferreira Soares,m,Bacharel em Direito.1898,1898-07-26,4.º e grau de Bacharel 26.07.1898 Aprovado Nem...,"Grijó, Vila Nova de Gaia",1897-06-16,None
174500,António Joaquim de Sá Oliveira,m,Bacharel em Direito.1898,1898-06-18,4.º e grau de Bacharel 18.06.1898 Aprovado Sim...,Arada,1894-10-24,None
168701,António Lino Neto,m,Bacharel em Direito.1898,1898-06-20,4º e grau de Bacharel 20.06.1898 Aprovado Nemi...,"Mação, Santarém",1894-10-12,Mação (Santarém)
211852,António Luís Vaz,m,Bacharel em Direito.1898,1898-06-25,4.º Bacharel 25.06.1898 Atos n.º 40 fl. 9,"Verdoejo, Valença",1891-10-13,None
217299,António Manuel Santiago,m,Bacharel em Direito.1898,1898-06-21,4.º e grau de Bacharel 21.06.1898 Aprovado Nem...,"Felgar, Moncorvo",1894-10-12,None
194072,António Pereira de Vasconcelos da Rocha Lacerda,m,Bacharel em Direito.1898,1898-06-21,"4.º ano e Grau de Bacharel: 21.06.1898, Aprova...",Ponte da Barca,1894-10-11,None
197166,António Xavier Abelho Laranjo,m,Bacharel em Direito.1898,1898-06-23,"4.º ano e Grau de Bacharel: 23.06.1898, Aprova...",Coimbra,1893-10-23,None


In [2]:
!pip install lxml

You should consider upgrading via the '/Users/jrc/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [3]:
from datetime import datetime

from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.sparql import return_sparql_query_results

from lxml import etree

# Prepare to fetch records from Portuguese National Library



xsl_file = '../extras/bnp/visbd-fauc.xsl'
xsl = etree.parse(xsl_file) 

# Things we would like to known about the students
claims_of_interest = ['P69','P1559','P1447','P27','P569','P570','P106','P19','P20',
                      'P22','P25','p97', 
                      'P973',                      # described at URL
                      'P1005','P102','P39','P166','P213','P214']
                      
claims_ignored = set()
prop_labels = {}
cache = {}
results = return_sparql_query_results(query)

# https://qwikidata.readthedocs.io/en/stable/entity.html
for result in results['results']['bindings']:
    uri = result['student']['value']
    qid = uri.split("/")[-1]  # last item in the path
    result_dict = get_entity_dict_from_api(qid)
    std = WikidataItem(result_dict)

    print(std.entity_id,std.get_label())
    for aka in std.get_aliases(lang='pt'):
        print(f"{' '*2} aka:",aka)

    claim_groups = std.get_claim_groups()


    claim_id_list = sorted([c for c in claim_groups.keys() if c in claims_of_interest])
    if 'P1005' in claim_id_list:     # we move the property related to published works to the end
        claim_id_list.remove('P1005')
        claim_id_list = claim_id_list + ['P1005']


    for claim_id in claim_id_list:
        if claim_id in prop_labels.keys():
            propLabel = prop_labels[claim_id]
        else:
            prop_dict = get_entity_dict_from_api(claim_id)
            propLabel = prop_dict['labels']['en']['value']
            prop_labels[claim_id] = propLabel

        claim_group = claim_groups[claim_id]
        print(f"{' '*2} {claim_id} {propLabel}({len(claim_group)})")
        for claim in claim_group:
            snak = claim.mainsnak
            if snak.snak_datatype == 'wikibase-item':
                qid = snak.datavalue.value["id"]
                entity = WikidataItem(get_entity_dict_from_api(qid))
                print(f"{' '*6}{qid} {entity.get_label(lang='pt')}")
            elif snak.value_datatype == 'string':
                print(f"{' '*6} {snak.datavalue.value}") 
            elif snak.value_datatype == 'time':
                svalue = snak.datavalue.value['time']
                try:
                    date_value = datetime.strptime(svalue,"+%Y-%m-%dT%H:%M:%SZ")
                    print(f"{' '*6} {date_value}")  
                except ValueError as e:
                    print("Problem converting date, probably date incomplete: "+svalue,e)
            else:
                snak_info = (claim.rank,snak.snaktype,snak.snak_datatype,snak.value_datatype)  
                print(f"{' '*6}{snak_info}")
            
            if claim_id == 'P1005': 
                bnp_id = snak.datavalue.value
                url = f"http://urn.bn.pt/bibliografia/unimarc/xml?id={bnp_id}"
                records = etree.parse(url)
                transform = etree.XSLT(xsl)
                print(str(transform(records)))

            for pid, quals in claim.qualifiers.items():
                prop = WikidataProperty(get_entity_dict_from_api(pid))
                for qual in quals:
                    if qual.snak.snaktype != "value":
                        continue
                    else:
                        print(f"{' '*9}{prop.get_label()} {qual.snak.datavalue}")

    print(f"{' '*3}Other properties:",set(claim_groups.keys())-set(claims_of_interest) )
    claims_ignored = claims_ignored.union(set(claim_groups.keys())-set(claims_of_interest))
    print()

print()
if len(claims_ignored) > 0:
    print("Properties present in the students and ignored in this list")
    for ignored in sorted(claims_ignored):
        if ignored in prop_labels.keys():
            propLabel = prop_labels[prop]
        else:
            prop_dict = get_entity_dict_from_api(ignored)
            propLabel = prop_dict['labels']['en']['value']
            prop_labels[prop] = propLabel  
        print(f"   {ignored:<6s}: {propLabel}")



Q61755108 Manuel Augusto Martins
   P106 occupation(1)
      Q82955 político
   P19 place of birth(1)
      Q2078801 Santa Maria Maior
   P20 place of death(1)
      Q25444 Funchal
   P27 country of citizenship(1)
      Q45 Portugal
   P569 date of birth(1)
       1867-04-10 00:00:00
   P570 date of death(1)
       1936-04-04 00:00:00
   P69 educated at(1)
      Q368643 Universidade de Coimbra
         academic major WikibaseEntityId(id=Q7748)
   Other properties: {'P735', 'P21', 'P1412', 'P18', 'P2671', 'P103', 'P31', 'P734'}

Q16917757 Francisco Correia de Mendonça
   P106 occupation(1)
      Q16533 juiz
   P19 place of birth(1)
      Q732548 Lagos
   P27 country of citizenship(1)
      Q45 Portugal
   P569 date of birth(1)
       1832-01-03 00:00:00
   P570 date of death(1)
       1899-08-04 00:00:00
   P69 educated at(1)
      Q368643 Universidade de Coimbra
   Other properties: {'P735', 'P21', 'P1412', 'P2671', 'P103', 'P31'}

Q17272838 Álvaro Rodrigues Azevedo
   aka: José Rodrig

## Lugares

---

## Places


#### Encontrar um lugar na wikipedia

---

### Find a place in Wikipedia

In [ ]:
!pip install mkwikidata


In [124]:
pquery_pt = """

SELECT DISTINCT ?location ?locationName ?instanceOf ?instanceOfLabel ?insideLabel ?inside2Label ?coordinates ?geonamesID WHERE {
                        ?location ?label '$place'@$lang .  
                        ?location wdt:P17 wd:Q45 .
                        ?location rdfs:label ?locationName .
                        ?location wdt:P31 ?instanceOf .
                        ?location wdt:P625 ?coordinates .


                        OPTIONAL {
                          ?location wdt:P131 ?inside .
                          ?inside wdt:P131 ?inside2 .
                          ?location wdt:P1566 ?geonamesID .
                        }
                        FILTER(lang(?locationName) = "$lang") 
                        { SERVICE wikibase:label { bd:serviceParam wikibase:language "$lang". }}
 
                        }
"""

In [190]:

pquery_world = """
SELECT DISTINCT ?location ?locationLabel ?altLabel ?locationName ?instanceOf ?instanceOfLabel ?insideLabel ?inside2 ?inside2Label ?coordinates ?geonamesID WHERE {
                        ?location ?label '$place'@$langsearch.  
                        ?location rdfs:label ?locationName  
                                FILTER (lang(?locationName) = "en" 
                                        || lang(?locationName) = "zh" 
                                        || lang(?locationName) = "pt"  ).
                        ?location wdt:P31 ?instanceOf .
                        ?location wdt:P131 ?inside .
                        ?inside wdt:P131 ?inside2 .
                        OPTIONAL {
                          ?location wdt:P625 ?coordinates .
                        }
                        OPTIONAL {
                          ?location wdt:P1566 ?geonamesID .
                        }
                        OPTIONAL { ?location skos:altLabel ?altLabel . 
                                      FILTER (lang(?altLabel) = "en" 
                                        || lang(?altLabel) = "zh" 
                                        || lang(?altLabel) = "fr" 
                                        || lang(?altLabel) = "pt"  ).
                        }
                        { SERVICE wikibase:label { bd:serviceParam wikibase:language "$langresult". }}
                        }

                        """

In [202]:
import mkwikidata
import pandas as pd

place = "Chansi"
lang_search = "en"
lang_result = 'en'
pquery = pquery_world

query_result = mkwikidata.run_query(pquery, params={'place':place,'langsearch':lang_search,'langresult':lang_result})
results = query_result['results']['bindings']
if len(results) > 0 :
    table = [{'name':loc['locationName']['value'],
            'nameLabel':loc['locationLabel']['value'],
            'alt':loc.get('altLabel',{'value':None})['value'],
            'type':loc['instanceOfLabel']['value'],
            'uri':loc['location']['value'],
            'inside':loc.get('insideLabel',{'value':None})['value'],
            'inside2':loc.get('inside2Label',{'value':None})['value'],
            'geonamesID':loc.get('geonamesID',{'value':None})['value'],
            'coordinates':loc.get('coordinates',{'value':None})['value'],

            } for loc in results]
    display(pd.DataFrame(table).sort_values('uri'))
else:
    print(f"Not found {place}")


Not found Chansi


In [ ]:
from timelinknb import current_time,current_machine, get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)
print(current_machine,current_time,f'db={db_spec}')

imac-jrc.local 2022-05-18 17:43:33.378466 db=('sqlite', 'fauc.db')


### Localizar os lugares mais infrequentes no FAUC

---

### Locate the less frequent places in the FAUC

In [ ]:
from timelinknb.pandas import attribute_values


# Get list of different values and the number of occurences
naturalidades = attribute_values('naturalidade')
small_places = naturalidades[naturalidades['count']<10].sort_values('count', ascending=False).head(100)

Test with a small number

In [ ]:
from IPython.display import display
import time

for place in small_places.index.values[:10]:
    print("Looking for:", place)
    try:
        query_result = mkwikidata.run_query(pquery, params={'place':place})
    except Exception as e:
        print(e)
    results = query_result['results']['bindings']
    if len(results) > 0:
        table = [{'name':loc['locationName']['value'],
                'type':loc['instanceOfLabel']['value'],
                'uri':loc['location']['value'],
                'inside':loc['insideLabel']['value'],
                'inside2':loc.get('inside2Label',{'value':None})['value'],
                'geonamesID':loc.get('geonamesID',{'value':None})['value'],
                
                } for loc in results]
        candidates = pd.DataFrame(table)
        display(candidates)
        geonames = candidates['geonamesID'].unique()
        for geo in geonames:
            print(f"   https://www.geonames.org/{geo}")
    else:
        print("   not found")
        
    time.sleep(1)  # respect wikidata

Looking for: Alcofra


,name,type,uri,inside,inside2,geonamesID
0,Alcofra,freguesia de Portugal,http://www.wikidata.org/entity/Q1021445,Vouzela,Viseu,8014009
1,Alcofra,freguesia de Portugal,http://www.wikidata.org/entity/Q1021445,Vouzela,Comunidade Intermunicipal Viseu Dão Lafões,8014009


   https://www.geonames.org/8014009
Looking for: Lumiar


,name,type,uri,inside,inside2,geonamesID
0,Estação Lumiar,estação subterrânea,http://www.wikidata.org/entity/Q374170,Lisboa,None,None
1,Estação Lumiar,estação de metropolitano,http://www.wikidata.org/entity/Q374170,Lisboa,None,None
2,Lumiar,freguesia de Portugal,http://www.wikidata.org/entity/Q924723,Lisboa,Lisboa,8012475


   https://www.geonames.org/None
   https://www.geonames.org/8012475
Looking for: Seixo de Ansiães


,name,type,uri,inside,inside2,geonamesID
0,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Trás-os-Montes e Alto Douro,8014442
1,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Bragança,8014442
2,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Douro,8014442


   https://www.geonames.org/8014442
Looking for: Santa Cristina de Figueiró


,name,type,uri,inside,inside2,geonamesID
0,Santa Cristina de Figueiró,localidade,http://www.wikidata.org/entity/Q1023987,Amarante,Porto,8012656
1,Santa Cristina de Figueiró,freguesia de Portugal,http://www.wikidata.org/entity/Q1023987,Amarante,Douro Litoral,8012656
2,Santa Cristina de Figueiró,localidade,http://www.wikidata.org/entity/Q1023987,Amarante,Douro Litoral,8012656
3,Santa Cristina de Figueiró,freguesia de Portugal,http://www.wikidata.org/entity/Q1023987,Amarante,Porto,8012656


   https://www.geonames.org/8012656
Looking for: Sandelgas
   not found
Looking for: Salvaterra
   not found
Looking for: Rio Bom
   not found
Looking for: Ribeira de Homem
   not found
Looking for: Quintãs
   not found
Looking for: Proença
   not found


## Biblioteca Nacional de Portugal

---

## Portuguese National Library



Get all the bibiographic records of a student with P1005 id.



In [94]:
from lxml import etree

bnp_id = "92687"
url = f"http://urn.bn.pt/bibliografia/unimarc/xml?id={bnp_id}"

xsl_file = '../extras/bnp/visbd-fauc.xsl'
xsl = etree.parse(xsl_file) 
records = etree.parse(url)
transform = etree.XSLT(xsl)

print(str(transform(records)))





Socialismo libertario e anarchismo : história e doutrina / Silva Mendes. - [S.l. : s.n.], 1896. - 369 p. ; 22 cm
http://id.bnportugal.gov.pt/bib/catbnp/418597

Sobre filosofia / Manuel da Silva Mendes. - Macau : Leal Senado, [1915]. - 169 p. : il. ; 21 cm
http://id.bnportugal.gov.pt/bib/catbnp/1579131

Socialismo libertário ou anarchismo : história e doutrina / Manuel da Silva Mendes. - ed. fac-simil. - Lisboa : Letra Livre, D.L. 2006. - 369 p. : fac-simil. ; 24 cm. - Ed. fac-simil., Lisboa, 1896. - ISBN 989-95114-0-4. - ISBN 978-989-95114-0-8
http://id.bnportugal.gov.pt/bib/catbnp/1563060

Manuel da Silva Mendes : a instrução pública em Macau ; org. e introd. António Aresta. - Macau : Direcção dos Serviços de Educação e Juventude, 1996. - 96, [5] p., [7] p. il. : il. ; 24 cm. - (Educação memórias ; 3). - ISBN 972-8091-32-X
http://id.bnportugal.gov.pt/bib/catbnp/987433

Arte chinesa : (colectânea de artigos) / Manuel da Silva Mendes. - [Macau] : Edição do Leal Senado de Macau, 1983. -

In [16]:
recs = records.getroot()
recs.tag

'{http://www.bn.pt/standards/metadata/marcxml/1.0/}collection'

In [92]:
marcxml_ns = "http://www.bn.pt/standards/metadata/marcxml/1.0/"
nsmap = {None: marcxml_ns}
for rec in recs:
    print("Record:")
    for cf in rec.findall("controlfield",namespaces=nsmap):
        print(cf.get('tag'),cf.text)
    for df in rec.findall("datafield",namespaces=nsmap):
        print(f"{df.get('tag'):3s} {df.get('ind1'):1s}{df.get('ind2'):1s}")
        for sf in list(df):
            print(f"   ${sf.get('code')} {sf.text}",)
print()


Record:
001 418597
003 http://id.bnportugal.gov.pt/bib/catbnp/418597
005 19981231000000.0
095   
   $a PTBN00447303
100   
   $a 19930417d1896    m  y0pory0103    ba
101 0 
   $a por
102   
   $a PT
200 0 
   $a Socialismo libertario e anarchismo
   $e história e doutrina
   $f Silva Mendes
210   
   $a [S.l.
   $c s.n.],
   $d 1896
215   
   $a 369 p.
   $d 22 cm
500 10
   $a Socialismo libertário ou anarquismo
700  1
   $a Mendes,
   $b Silva,
   $f 1867-1931
   $3 92687
801  0
   $a PT
   $b BN
   $g RPC
966   
   $l BN
   $m FGMON
   $s S.C. 3676 V.
966   
   $l BN
   $m FGMON
   $s S.C. 11055 P.
997   
   $a BIBEOD
Record:
001 1579131
003 http://id.bnportugal.gov.pt/bib/catbnp/1579131
100   
   $a 20070321d1915    m  y0pory0103    ba
101 0 
   $a por
102   
   $a MO
105   
   $a a   z   001yy
106   
   $a r
200 1 
   $a Sobre filosofia
   $f Manuel da Silva Mendes
210   
   $a Macau
   $c Leal Senado,
   $d [1915]
215   
   $a 169 p.
   $c il.
   $d 21 cm
675   
   $a 22Taoísmo
  

In [10]:
recs = records['collection']

cfields = recs['record'][0]['controlfield']
dfields = recs['record'][0]['datafield']
cfields


TypeError: 'lxml.etree._ElementTree' object is not subscriptable